 # Safe to eat or deadly poisonous?
 ### An analysis on mushroom classification by Lorenzo Santolini

 ### Index:
   1. [Introduction](#introduction)
   2. [Dataset Analysis](#dsl)
   3. [Preprocessing](#preprocessing)
   4. [Principal Component Analysis](#pca)
   5. [Classification](#classification)
   6. [Conclusions](#conclusions)

 This is a little code to import automatically the dataset into google colab.
 Provide your kaggle's API key (profile section) when file requested
 ### Code snippet for google colab

In [1]:
# Little code snippet to import on Google Colab the dataset
'''
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

# Insert here your kaggle API key
from google.colab import files
files.upload()

!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d uciml/mushroom-classification
!unzip mushroom-classification.zip
!ls
'''


'\n!pip install -U -q kaggle\n!mkdir -p ~/.kaggle\n\nfrom google.colab import files\nfiles.upload()\n\n!cp kaggle.json ~/.kaggle/\n!chmod 600 /root/.kaggle/kaggle.json\n!kaggle datasets download -d uciml/mushroom-classification\n!unzip mushroom-classification.zip\n!ls\n'

In [2]:
# Define all the constants that will be used

PLOTLY_COLORS = ['#140DFF', '#FF0DE2']
COLOR_PALETTE = ['#140DFF', '#FF0DE2', '#CAFFD0', '#C9E4E7', '#B4A0E5', '#904C77']
COLORSCALE_HEATMAP = [         [0.0, 'rgb(70,0,252)'], 
                [0.1111111111111111, 'rgb(78,0,252)'], 
                [0.2222222222222222, 'rgb(90,0,252)'], 
                [0.3333333333333333, 'rgb(110,0,248)'], 
                [0.4444444444444444, 'rgb(130,0,238)'], 
                [0.5555555555555556, 'rgb(145,0,228)'], 
                [0.6666666666666666, 'rgb(166,0,218)'], 
                [0.7777777777777778, 'rgb(187,0,213)'], 
                [0.8888888888888888, 'rgb(200,0,202)'], 
                               [1.0, 'rgb(210,0,191)']]
PLOTLY_OPACITY = 0.7
RANDOM_SEED = 11

LOGISTIC_REGRESSION_PARAMS = [{
    'clf__solver': ['liblinear'],  # best for small datasets
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], # smaller value, stronger regularization, like svm
    'clf__penalty': ['l2', 'l1']
},
{
    'clf__solver': ['newton-cg', 'lbfgs'], 
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l2'] # `newton-cg` and `lbfgs` accept only l2
}]

SVM_PARAMS = [
{
    'clf__kernel': ['linear'],
    'clf__C': [0.1, 1, 10, 100],
}, 
{
    'clf__kernel': ['rbf'],
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': [0.1, 1, 10, 100],
}]

RANDOM_FOREST_PARAMS = {
    'clf__max_depth': [50, 75, 100],
    'clf__max_features': ["sqrt", "log2"], # sqrt is the same as auto
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 300, 500]
}

KNN_PARAMS = {
    'clf__n_neighbors': [2, 3, 5, 15, 25],
    'clf__weights': ['uniform', 'distance'],
    'clf__p': [1, 2, 10]
}



 <a id='introduction'></a>
# Introduction
 This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom. Each species is identified as edible or poisonous. Rows are composed by 23 different fields, each one of them identifying a specific charateristic:
 - `Class`: poisonous=p, edible=e
 - `Cap-surface`: fibrous=f, grooves=g, scaly=y, smooth=s
 - `Cap-shape`: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
 - `Cap-color`: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
 - `Bruises`: bruises=t, no=f
 - `Odor`: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
 - `Gill-attachment`: attached=a, descending=d, free=f, notched=n
 - `Gill-spacing`: close=c, crowded=w, distant=d
 - `Gill-size`: broad=b, narrow=n
 - `Gill-color`: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u,red=e, white=w, yellow=y
 - `Stalk-shape`: enlarging=e, tapering=t
 - `Stalk-root`: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
 - `Stalk-surface-above-ring`: fibrous=f, scaly=y, silky=k, smooth=s
 - `Stalk-surface-below-ring`: fibrous=f, scaly=y, silky=k, smooth=s
 - `Stalk-color-above-ring`: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
 - `Stalk-color-below-ring`: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
 - `Veil-type`: partial=p, universal=u
 - `Veil-color`: brown=n, orange=o, white=w, yellow=y
 - `Ring-number`: none=n, one=o, two=t
 - `Ring-type`: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
 - `Spore-print-color`: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
 - `Population`: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
 - `Habitat`: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

 This analysis was conducted in *Python 3.7.1* using Jupyter Notebook allows you to combine code,
 comments, multimedia, and visualizations in an interactive document — called a notebook,
 naturally — that can be shared, re-used, and re-worked.

 In addition, the following packages were used:
 - sklearn
 - pandas
 - numpy
 - plotly
 - scipy
 - prettytable
 - imblearn

In [3]:
# Import all the libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, learning_curve, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle

import plotly
import plotly.plotly as py
from plotly.plotly import plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

from scipy.cluster import hierarchy as hc
import scipy.spatial as scs

from imblearn.pipeline import make_pipeline, Pipeline

import warnings
from collections import defaultdict
from prettytable import PrettyTable
from functools import wraps
import time

plotly.tools.set_credentials_file(username='modusV', api_key='OBKKnTR2vYTeKIOKtRU6')
warnings.filterwarnings("ignore")

In [4]:

# Wrapper to calculate functions speed

def watcher(func):
    """
    Shows how much time it
    takes to execute function
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f" ===> took {end-start} seconds")
        return result
    return wrapper


 <a id='dsl'></a>
# Dataset load and overall view
 Let's start importing the data:

In [5]:
# Load the dataset
dataset = pd.read_csv("./Input/mushrooms.csv")
# dataset = pd.read_csv("./mushrooms.csv")


In [6]:
# Shape of the dataset
print("The dataset has %d rows and %d columns." % dataset.shape)


The dataset has 8124 rows and 23 columns.


 We will look now at the dataset to understand what are the different fields and their types:

In [7]:
# Count number of classes for classification
print(f"There are {dataset['class'].unique().size} different classes:"
      f"\n {dataset['class'].unique().tolist()}")

# Count number of unique data for every column
print(f"Unique values for every field: \n{dataset.nunique()}")


There are 2 different classes:
 ['p', 'e']
Unique values for every field: 
class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64


 We can notice that `veil-type` has just one value, therefore that column is useless
 for our analysis. We will remove it later.

 We can now look deeper inside the dataset. Thanks to the pandas library, we can
 see all the fields of the dataset with the respective values.

 <a id='preprocessing'></a>
# Preprocessing
 Before starting the classification phase, we need to preprocess the dataset, in
 such a way that our classifiers will score with more accuracy and reliability.
 This is the most important step, if data are messy the classification will perform
 poorly.
 The steps that we will go trough are:

 1. Check data types
 2. Remove not significat columns, if any
 3. Remove null values, if any
 4. Encode string values
 5. Check class distribution and, if classes are unbalanced, apply balancing techniques
 6. Check data distribution using of bar graphs and box plots
 7. Analyze correlation matrix to understand which fields are more important to classify our samples
 8. Divide the dataset in classes array and unclassified samples
 9. Scale our data, in such a way to center and standardize them

 ### 1 - Check data types

In [8]:
# See data types 
print(f"Data types: \n{dataset.head(5)}")


Data types: 
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color   ...   stalk-surface-below-ring  \
0            c         n          k   ...                          s   
1            c         b          k   ...                          s   
2            c         b          n   ...                          s   
3            c         n          n   ...                          s   
4            w         b          k   ...                          s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w  

 From the above snippet we can notice that the fields are all string values;
 converting them to numeric values will make our analysis much easier. We will use a
 library called `LabelEncoder`. It allows us with a few line of code to create a mapping
 of every value in each field and transform the data in this way. We can go back to the
 original mapping simply using the `inverse_transform` function.

 Before this step though, we will firstly remove all the useless columns, in this
 case just `veil-type`.

 ### 2 - Remove any not significant column
 Now we will remove all the fields that do not add any information to our analysis,
 specifically, the fields that contain only one value (zero variance).

In [9]:
n_columns_original = len(dataset.columns)
to_drop = [col for col in dataset.columns if dataset[col].nunique() == 1]
dataset.drop(to_drop, axis=1, inplace=True)

for d in to_drop:
    print(str(d) + " ", end="")
print("have been removed because they have zero variance")
print(f"{n_columns_original - len(dataset.columns)} not significant columns have been removed")


veil-type have been removed because they have zero variance
1 not significant columns have been removed


 As we can notice, only one field was removed.

 ### 3 - Handling missing values
 When we find any missing value in a dataset, there are different
 approaches that can be considered:

 1. Delete all rows containing a missing value
 2. Substitute with a constant value that has meaning within the domain, such as 0, distinct from all other values.
 3. Substitute with a value from another randomly selected record.
 4. Substitute with mean, median or mode value for the column.
 5. Substitute with a value estimated by another predictive model.


 We will approach this problem using the first and the second techniques:

 1. We will create a parallel dataset in which all the rows containing a missing
 value will be dropped, classifying
 them as incomplete samples. This may cause a large decrement in the dataset size
 but is the only way
 to be sure that we are not going to influence our classification algorithm in any
 way.

 2. It is evident from the `dataset.head()` function that our fileds are composed
 by all string values.
 Given the fact that we would need to translate in any case every field to a
 numeric one, to better display
 them in graphs, a simple approach is to keep the missing data as a peculiar number
 different from the others,
 and simply apply the transformation as they were present.


 In any case, let's start counting how many null/missing values we will find.

In [10]:
# Check if any field is null
if dataset.isnull().any().any():
    print("There are some null values")
else:
    print("There are no null values")

There are no null values


 It may seem that we have no missing value from the previous analysis... Great!

 But wait a minute ... If we look better,from the data description we can notice that in the field
 `stalk-root` there are some missing values, marked with the question mark; let's count how many of them there are:

In [11]:
print("There are " + str((dataset['stalk-root'] == "?").sum()) + " missing values in stalk-root column")


There are 2480 missing values in stalk-root column


 More than 25% of our samples is incomplete. Dropping all those rows may lead to a shortage of samples.
 This is why we will use the two approaches and see which one performs better.
 In any case, we will replace those value with another character.
 We will use the character `m` to indicate a missing value

In [12]:

dataset['stalk-root'] = dataset['stalk-root'].replace({"?": "m"})
print(dataset['stalk-root'].unique())


['e' 'c' 'b' 'r' 'm']


 ### 4 - Encode string values
 As already said, we need to encode all the string values into integers, in such a way to continue our analysis in a more easy way.

In [13]:

def encode_values(dataset):
    """
    Encode string values of a dataset using numbers

    :param (array of arrays) dataset: Input dataset to encode
    """
    mapping = {}  
    d = dataset.copy()
    labelEncoder = LabelEncoder()
    for column in dataset.columns:
        labelEncoder.fit(dataset[column])
        mapping[column] = dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
        d[column] = labelEncoder.transform(dataset[column])
        
    return d, labelEncoder, mapping

def print_encoding(mapping):
    """
    Prints a table with the key-value corrispondence of an encoding

    :param (dict) mapping: dictionary containing value before and after encoding
    """
    t = PrettyTable()
    rows = []
    for key, value in mapping.items():
        r = []
        r.append(key)
        for k, v in value.items():
            r.append(k)
        rows.append(r)
    max = []
    for r in rows:
        if len(r) > len(max):
            max = r

    for r in rows:
        r = r + ['-'] * (len(max) - len(r))
        t.add_row(r)
    t.field_names = ["Columns / Values"] + list(range(0, len(max)-1))
    print(t)

le = 0
pre_data, l_encoder, le_mapping = encode_values(dataset)

# Check mapping
print_encoding(le_mapping)

# Check new data
pre_data.head(5)


+--------------------------+---+---+---+---+---+---+---+---+---+---+----+----+
|     Columns / Values     | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 |
+--------------------------+---+---+---+---+---+---+---+---+---+---+----+----+
|          class           | e | p | - | - | - | - | - | - | - | - | -  | -  |
|        cap-shape         | b | c | f | k | s | x | - | - | - | - | -  | -  |
|       cap-surface        | f | g | s | y | - | - | - | - | - | - | -  | -  |
|        cap-color         | b | c | e | g | n | p | r | u | w | y | -  | -  |
|         bruises          | f | t | - | - | - | - | - | - | - | - | -  | -  |
|           odor           | a | c | f | l | m | n | p | s | y | - | -  | -  |
|     gill-attachment      | a | f | - | - | - | - | - | - | - | - | -  | -  |
|       gill-spacing       | c | w | - | - | - | - | - | - | - | - | -  | -  |
|        gill-size         | b | n | - | - | - | - | - | - | - | - | -  | -  |
|        gill-color        | b | e | g | h | k | n |

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1


 As we can see data have been transformed; all the strings values now are equal to integers,
 and we can see the direct corrispondence from the table above.

 We have encoded our dataset but, depending on the data (as in our case), label encoding may
 introduce a new problem. For example, we have encoded a set of colour names
 into numerical data. This is actually categorical data and there is no relation,
 of any kind, between the rows.

 The problem here is, since there are different numbers in the same column,
 the model will misunderstand the data to be in some kind of order, 0 < 1 < 2.
 But this isn’t the case at all. To overcome this problem, we use `OneHotEncoder`.

 What one hot encoding does is, it takes a column which has categorical data,
 which has been label encoded, and then splits the column into multiple columns.
 The numbers are replaced by 1s and 0s, depending on which column has what value.

 We will obtain two datasets; we will continue the analysis on the first one, but
 meanwhile we will keep this one for the classification phase, to check if we
 have an improvement with respect to the other.

 We also drop the column with missing value, beacuse it is not significative.

In [14]:

def one_hot_encode(X_dataset):

    ohc = defaultdict(OneHotEncoder)
    d = defaultdict (LabelEncoder)
    Xfit = X_dataset.apply(lambda x: d[x.name].fit_transform(x))
    final = pd.DataFrame()

    for i in range(len(X_dataset.columns)):
        # Transform columns using OneHotEncoder
        Xtemp_i = pd.DataFrame(ohc[Xfit.columns[i]].fit_transform(Xfit.iloc[:,i:i+1]).toarray())
    
        # Naming the columns
        ohc_obj  = ohc[Xfit.columns[i]]
        labelEncoder_i= d[Xfit.columns[i]]
        Xtemp_i.columns= Xfit.columns[i]+"-"+labelEncoder_i.inverse_transform(ohc_obj.active_features_)
        
        # Take care of dummy variable trap dropping of new coulmns
        X_ohc_i = Xtemp_i.iloc[:,1:]
        
        # Append columns to dataframe
        final = pd.concat([final,X_ohc_i],axis=1)

    return final

pre_ohc_data = one_hot_encode(dataset.iloc[:,1:])
pre_ohc_data.drop(['stalk-root-m'], axis=1, inplace=True)
pre_ohc_data.head(5)



,cap-shape-c,cap-shape-f,cap-shape-k,cap-shape-s,cap-shape-x,cap-surface-g,cap-surface-s,cap-surface-y,cap-color-c,cap-color-e,...,population-n,population-s,population-v,population-y,habitat-g,habitat-l,habitat-m,habitat-p,habitat-u,habitat-w
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


 As we can see, we obtained a huge dataset, where all the fields are binary.

 ### 5/6 - Check class and data distribution
In this phase, we will analyze the distribution of the data. The steps will be:

 1. Check amount of samples belonging to a class or to another.
 2. Analyze the overall distribution using box plots, a very useful tool to identify outliers and values taken by samples.
 3. Compare the two classes distributions with the help of an histogram.

 #### 5 - Check classes distribution
 Let's see how many samples belong to the different classes

In [15]:
y = dataset["class"].value_counts()
print(y)
class_dict = ["edible", "poisonous"]


e    4208
p    3916
Name: class, dtype: int64


 Luckily the dataset is pretty balanced:
 We have almost the same amount of samples in a class and in the other; this simplifies the analysis because we can assign the same weight to the two classes in the classification phase.

 Moreover, we can notice that the classification task will be binary. Infact, data have been transformed, and now the labels are represented with a 0/1 integer value.
 Now we can look deeper into some statistical details about the dataset, using the `pre_df.describe()` command on our pandas DataFrame dataset. The output shows:

 - count: number of samples (rows)
 - mean: the mean of the attribute among all samples
 - std: the standard deviation of the attribute
 - min: the minimal value of the attribute
 - 25%: the lower percentile
 - 50%: the median
 - 75%: the upper percentile
 - max: the maximal value of the attribute

In [16]:
# Get insights on the dataset
pre_data.describe()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,...,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,...,1.575086,1.603644,5.816347,5.794682,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,0.499708,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,...,0.621459,0.675974,1.901747,1.907291,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,...,1.000000,1.000000,6.000000,6.000000,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,0.000000,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,...,2.000000,2.000000,7.000000,7.000000,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,1.000000,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,...,2.000000,2.000000,7.000000,7.000000,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,...,3.000000,3.000000,8.000000,8.000000,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


 This is the class distribution plot on a graph bar. As we already saw before the class distribution is pretty balanced.

 In this report, all the graphic part will use the `plotly` library.

 *plotly.py* is an interactive, open-source, and browser-based graphing library for Python, which allows you to create interactive plots in a few steps.

In [17]:
data = [go.Bar(
            x=class_dict,
            y=y,
            marker=dict(
            color=PLOTLY_COLORS),
            opacity=PLOTLY_OPACITY,
    )]

layout = go.Layout(title="Class distribution",
                   autosize=True,
                   yaxis=dict(
                        title='N. samples',
                    ),
                   )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='distribution-bar')


 #### 6 - Box plot

 At this point we can analyze the distribution of our data using a boxplot.
 A boxplot is a standardized way of displaying the distribution of data based on a five number summary (“minimum”, first quartile (Q1), median, third quartile (Q3), and “maximum”).

 It can tell you about your outliers and what their values are.
 It can also tell you if your data is symmetrical, how tightly your data is grouped,and if and how your data is skewed.
 The information that we can find in a box plot are:

 - **median** (Q2/50th Percentile): the middle value of the dataset.
 - **first quartile** (Q1/25th Percentile): the middle number between the smallest number (not the “minimum”) and the median of the dataset.
 - **third quartile** (Q3/75th Percentile): the middle value between the median and the highest value (not the “maximum”) of the dataset.
 - **interquartile range** (IQR): 25th to the 75th percentile.
 - **outliers** (shown as green circles)
 - **maximum**: Q3 + 1.5*IQR
 - **minimum**: Q1 -1.5*IQR

 It makes no sense showing binary or with few different values fields, so we are going to filter them before plotting.

In [18]:

def create_box(type, data, col, visible=False):
    if type == "edible":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    return go.Box(
        y = data[col],
        name = type,
        marker=dict(color = c),
        visible=visible,
        opacity=PLOTLY_OPACITY,
    )

edible = pre_data[pre_data["class"] == 0]
poisonous = pre_data[pre_data["class"] == 1]
box_features = [col for col in pre_data.columns if ((col != 'class') and (dataset[col].nunique() > 5))]

active_index = 0
box_edible = [(create_box("edible", edible, col, False) if i != active_index 
               else create_box("edible", edible, col, True)) 
              for i, col in enumerate(box_features)]

box_poisonous = [(create_box("poisonous", poisonous, col, False) if i != active_index 
               else create_box("poisonous", poisonous, col, True)) 
              for i, col in enumerate(box_features)]

data = box_edible + box_poisonous
n_features = len(box_features)
steps = []

for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = box_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    autosize=True,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='box_slider')


 From the boxplot above, we can see that the color and the shape of the
 cap are not an effective parameter to decide whether a mushroom is
 poisonous or edible, because their plots are very similar (same median
 and very close distribution).
 The `odor` and the `population` columns, on the other hand, are more significant;

 In the `odor` field, all the edible mushrooms are squeezed into a single value
 with a few outliers, while the poisonous may have all the different values.

 #### 6 - Bar graph

 A bar chart or bar graph is a chart or graph that presents categorical data with rectangular bars with heights or lengths proportional to the values that they represent.
 With a slider we can move along the different features, to better visualize the value distributions.

In [19]:

def create_bar(type, data, col, visible=False):
    if type == "edible":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    return go.Histogram(
        x = data[col],
        name = type,
        marker=dict(color = c),
        visible=visible,
        opacity=PLOTLY_OPACITY,
    )

hist_features = [col for col in pre_data.columns if (col != 'class')]

active_index = 0
hist_edible = [(create_bar("edible", edible, col, False) if i != active_index 
               else create_bar("edible", edible, col, True)) 
              for i, col in enumerate(hist_features)]

hist_poisonous = [(create_bar("poisonous", poisonous, col, False) if i != active_index 
               else create_bar("poisonous", poisonous, col, True)) 
              for i, col in enumerate(hist_features)]

total_data = hist_edible + hist_poisonous
n_features = len(hist_features)
steps = []

for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(total_data)],
        label = hist_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    autosize=True,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = dict(data=total_data, layout=layout)
py.iplot(fig, filename='bar_slider')


 As we saw from the box plot, also here we can notice that
 the `cap-shape` and the `cap-color` are not that significant to classify a sample.
 On the other hand, some fields like `odor` show a distinct
 separation of the two classes; these ones will be the ones with more
 impact on our classification algorithms.

 From these bar graphs we can see that our dataset is pretty well
 separated. This means that, in our classification task, we will be
 able to achieve high accuracy even with some dimensionality reduction.

 ### 7 - Correlation matrix
 A correlation matrix is a table showing correlation coefficients between sets of variables.
 Each random variable (Xi) in the table is correlated with each of the other values in the table (Xj).
 This allows you to see which pairs have the highest correlation. Correlation is any statistical association, though in common usage it most often refers to how close two variables are to having a linear relationship with each other.

 We will use the **Pearson's correlation**, which is a measure of the linear correlation between two variables X and Y. According to the Cauchy–Schwarz inequality it has a value between +1 and −1, where 1 is total positive linear correlation, 0 is no linear correlation, and −1 is total negative linear correlation.

 The coefficient for a population is computed as:

 $\rho_{(X,Y)} = \frac{cov(X,Y)}{\sigma_X\sigma_Y}$

 Where:
 - $cov$ is the covariance:
    $cov(X,Y) = \frac{E[(X - \mu_X)(Y-\mu_Y)]}{\sigma_X\sigma_Y}$
    - Where $\mu$ is the mean value

 - $\sigma_X$ is the standard deviation of X
     $\sigma_X^2 = E[X^2] - [E[X]]^2$
 - $\sigma_Y$ is the standard deviation of Y
    $\sigma_Y^2 = E[Y^2] - [E[Y]]^2$

In [20]:
correlation_matrix = pre_data.corr(method='pearson')

trace = go.Heatmap(
    z=correlation_matrix.values.tolist(), 
    x=correlation_matrix.columns, 
    y=correlation_matrix.columns, 
    colorscale=COLORSCALE_HEATMAP,
    opacity=0.95,
    zmin=-1,
    zmax=1)
    

data=[trace]

layout = go.Layout(
    title='Heatmap of columns correlation',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap4')


 From the matrix, we can see that the most correlated columns to
 the `class` field are `gill-color`, `gill-size` and `bruises`.

 The diagonal has correlation 1 because every class has maximum correlation with itself.

 We can also see that `veil-color` and `gill-attachment` are highly correlated.

 ### 7 - Dendogram
 A **dendrogram** is a diagram representing a tree. This diagrammatic representation is frequently used in different contexts, but we will see the case representing hierarchical clustering.

 It illustrates the arrangement of the clusters, and its objective is to analyze if we have any duplicate features. In order to reduce the dimensionality of our dataset, we can identify and remove duplicate features according to their pairwise correlation with others.

 The linkage criterion determines the distance between sets of observations as a function of the pairwise distances between observations.
 We will use the between-group average linkage (UPGMA). Proximity between two clusters is the arithmetic mean of all the proximities between the objects of one, on one side, and the objects of the other, on the other side.
 The method is frequently set the default one in hierarhical clustering packages.

In [21]:

names = pre_data.columns
inverse_correlation = 1 - abs(pre_data.corr()) # This is the 'dissimilarity' method

fig = ff.create_dendrogram(inverse_correlation.values, 
                           labels=names, 
                           colorscale=COLOR_PALETTE, 
                           linkagefun=lambda x: hc.linkage(x, 'average'))

fig['layout'].update(dict(
    title="Dendrogram of correlation among features",
    width=800, 
    height=600,
    xaxis=dict(
        title='Features',
    ),
    yaxis=dict(
        title='Distance',
        
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')


 From the above graph, the closest features are `veil-color` and `gill-attachment`.
 Because that their distance is still far from zero, I choose not to remove any of the two features; moreover our dataset is pretty small, so we should not have great performance issues.

 ### 8/9 - Scale and divide data
 Most of the times, datasets contain features highly varying in magnitudes, units and range.
 But since, most of the machine learning algorithms use Eucledian distance between two data points in their computations, this is a problem. If left alone, these algorithms only take in the magnitude of features neglecting the units. The results would vary greatly between different units, for example between 5kg and 5000gms.

 The features with high magnitudes will weigh in a lot more in the distance calculations than features with low magnitudes.
 To supress this effect, we need to bring all features to the same level of magnitudes. This can be acheived by scaling.

 We will use the `StandardScaler`, which standardizes our data both with mean and standard deviation.

 The operation performed will be:

 $$
 x' = \frac{x - \mu_x}{\sigma_x}
 $$
 After this step, we divide the dataset into an array of unclassified samples and an array of labels, to use for the classification phase.

 At the end, we decide to bring to the next step of our analysis two
 different datasets, plus the one that we created in the encoding phase.

 1. The full dataset, where the missing values are encoded with an integer.
 2. A reduced version of the dataset, where the rows with missing data are considered as incomplete samples and are dropped.

In [22]:

def dataframe_to_array(data):
    y_data = data['class']
    X_data = data.drop(['class'], axis=1)
    return X_data, y_data

def scale_data(X_data):
    """

    """
    scaler = StandardScaler(with_mean=True, with_std=True, copy=True)
    return scaler.fit_transform(X_data)


In [23]:

drop_data = pre_data[pre_data['stalk-root'] != le_mapping['stalk-root']['m']]
data_no_stalk = pre_data.drop(['stalk-root'], axis=1)

X_pre_data, y_data = dataframe_to_array(pre_data)
X_scaled_data = scale_data(X_pre_data)

X_drop_data, y_drop_data = dataframe_to_array(drop_data)
X_scaled_drop_data = scale_data(X_drop_data)

X_no_stalk, y_no_stalk = dataframe_to_array(data_no_stalk)
X_scaled_no_stalk = scale_data(X_no_stalk)


 <a id='pca'></a>
 ## Principal component analysis
 When our data are represented by a matrix too large (the number of dimensions is too high), it is difficult to extract the most interesting features and find correlations among them; moreover the space occupied is very high. PCA is a technique that allows to achieve dimensionality reduction while preserving the most important differences among samples.

 This transformation is defined in such a way that the first principal component has the largest possible variance (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components. The resulting vectors (each being a linear combination of the variables and containing n observations) are an uncorrelated orthogonal basis set.

 Let's calculate the Principal Components and show their retained variance on a bar graph.

In [24]:
def plot_cumulative_variance(pca):
    """
    Plots cumulative variance of all PC

    :param (pca object) pca: Pca object
    """   

    tot_var = np.sum(pca.explained_variance_)
    ex_var = [(i / tot_var) * 100 for i in sorted(pca.explained_variance_, reverse=True)]
    cum_ex_var = np.cumsum(ex_var)

    cum_var_bar = go.Bar(
        x=list(range(1, len(cum_ex_var) + 1)), 
        y=ex_var,
        name="Variance of each component",
        marker=dict(
            color=PLOTLY_COLORS[0],
        ),
        opacity=PLOTLY_OPACITY
        )

    variance_line = go.Scatter(
        x=list(range(1, len(cum_ex_var) + 1)),
        y=cum_ex_var,
        mode='lines+markers',
        name="Cumulative variance",
        marker=dict(
            color=PLOTLY_COLORS[1],
        ),
        opacity=PLOTLY_OPACITY,
        line=dict(
            shape='hv',
        ))
    data = [cum_var_bar, variance_line]
    layout = go.Layout(
        title='Individual and Cumulative Explained Variance',
        autosize=True,
        yaxis=dict(
            title='Explained variance (%)',
        ),
        xaxis=dict(
            title="Principal components",
            dtick=1,
            rangemode='nonnegative'
        ),
        legend=dict(
            x=0,
            y=1,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename='basic-bar')

def compress_data(X_dataset, n_components, plot_comp=False):
    
    """
    Performs pca reduction of a dataset.

    :param (array of arrays) X_dataset: Dataset to reduce
    :param (int) n_components: N components to project on 
    :param (bool) plot_comp: Plot explained variance

    :returns (pandas dataframe) X_df_reduced: pandas dataframe with reduced dataset
    :return (iplot) p: Plot, only if plot_com equals True. To plot the graph,
                       simply call the return value.
    """   

    pca = PCA(random_state=RANDOM_SEED)
    projected_data = pca.fit_transform(X_dataset)

    if plot_comp:
        p = plot_cumulative_variance(pca)

    pca.components_ = pca.components_[:n_components]
    reduced_data = np.dot(projected_data, pca.components_.T)
    X_df_reduced = pd.DataFrame(reduced_data, columns=["PC#%d" % (x + 1) for x in range(n_components)])
    if plot_comp:
        return p, X_df_reduced
    else:
        return X_df_reduced
    

In [25]:

plot, X_df_reduced = compress_data(X_dataset=X_scaled_data,
                             n_components=9,
                             plot_comp=True)
plot                       


 From the graph we can see that the first 9 components retain almost 80% of total variance,
 while last 5 not even 2%. We then choose to select first nine of them.

 This allows us to work on a smaller dataset achieving similar results,
 because most of the information is maintained.

 In the function `compress_data`, the dataset is also projected on these vectors,
 in such a way to obtain data reduction. This is a simple scalar product:

In [26]:

X_df_drop_reduced = compress_data(X_dataset=X_scaled_drop_data,
                                  n_components=9,
                                  plot_comp=False)

plot, X_df_ohc_reduced = compress_data(X_dataset=pre_ohc_data,
              n_components=20,
              plot_comp=True)
plot

 We can see that on those many components, some of them can be excluded. On this dataset we will keep
 the 80% variance rule, so I decided to project data on the first 20 components.

In [27]:
X_df_reduced.head(4)


,PC#1,PC#2,PC#3,PC#4,PC#5,PC#6,PC#7,PC#8,PC#9
0,-0.833526,0.371394,0.635124,-0.979212,0.805091,0.214213,0.822674,-1.221462,0.388396
1,-0.021193,1.422777,1.818224,0.924143,0.497426,-0.516796,-1.538924,-1.047926,0.303515
2,-0.920911,0.600343,0.548433,0.981545,-0.516677,-0.344558,-1.032707,-1.093875,0.995428
3,-1.077559,0.165357,0.947636,-0.681089,0.432045,-0.437544,0.675030,-1.603679,0.451185



 We will try now to use a scatte-plot to show if a clustering algorithm applied
 on the first two principal components is able to separate the samples in two clusters.

 PCA tries to find combinations of features that lead to maximum separation between data points.
 What this means is that, if we had a dimension in our dataset which was the same for all members,
 then that would not be considered, alone or in combination, among the top principal components.
 Only the features that vary a lot from data point to data point form a part of the top principal components.
 As a result, the points should appear to be quite far apart from each other on the plot.

 The plot of the PCA clusters may not make sense in a cuple of conditions:

 1. There is a lot of variance in the dataset, so the first two components
 cannot significantly represent the data
 2. The clustering algorithm focuses on features considered unimportant by the
 PCA.

 Our dataset should not have that much variance, so the clustering
 algorithm should be able to decently separate data in two clusters.


In [28]:

values = pre_data.values
pca = PCA(n_components=2)
x = pca.fit_transform(values)

kmeans = KMeans(n_clusters=2, random_state=RANDOM_SEED)
X_clustered = kmeans.fit_predict(values)

print(len(np.where(X_clustered == 0)[0]))
print(len(np.where(X_clustered == 1)[0]))

c1_idx = np.where(X_clustered == 0)
c2_idx = np.where(X_clustered == 1)

p1 = go.Scatter(
    x=np.take(x[:,0], indices=c1_idx)[0],
    y=np.take(x[:,1], indices=c1_idx)[0],
    mode='markers',
    name="Cluster1",
    marker=dict(
        color=PLOTLY_COLORS[0],
    ),
    opacity=PLOTLY_OPACITY)

p2 = go.Scatter(
    x=np.take(x[:,0], indices=c2_idx)[0],
    y=np.take(x[:,1], indices=c2_idx)[0],
    mode='markers',
    name="Cluster2",
    marker=dict(
        color=PLOTLY_COLORS[1],
    ),
    opacity=PLOTLY_OPACITY)
    

data = [p1, p2]

layout = go.Layout(
    title='Data clustered using first two components',
    autosize=True,
    yaxis=dict(
        title='Second component',
    ),
    xaxis=dict(
        title="First component",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='clusters-scatter')

6180
1944


 As expected, using K-means we are able to separate two groups of data using the two components with maximum variance.


 <a id='classification'></a>
# Classification
 Now we are going to explore different supervised learning classification methods, and see in the end the one that performs better.

 Now, before starting the classification phase, let's see what kind of pre-processed data it is better to use to achieve the best classification possible.
 Due to the fact that our dataset is pretty small, probably the dimensionality reduction using PCA is not strictly necessary, but if we can achieve a similar score using just main principal components, it is definitely better.

 We are going to compare results of several classification methods on the different datasets. In this way we can choose the one
 to pick for the next phase.
 The current versions of the dataset are:

 1. Full dataset.
 2. Reduced dataset by means of PCA.
 3. Dataset with missing values removed.
 4. Dataset with missing values removed reduced using PCA
 5. Dataset with column containing missing values removed
 6. Dataset encoded with OneHotEncoder
 7. Dataset encoded with OneHotEncoder compressed using PCA

 Our dataset is pretty balanced, so we do not strictly need any over or under-sampling technique.
 Let's start with splitting the datasets in train and test.

In [29]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled_data, y_data, test_size=0.2, random_state=RANDOM_SEED)
X_train_pc, X_test_pc, y_train_pc, y_test_pc = train_test_split(X_df_reduced, y_data, test_size=0.2, random_state=RANDOM_SEED)

X_train_drop, X_test_drop, y_train_drop, y_test_drop = train_test_split(X_scaled_drop_data, y_drop_data, test_size=0.2, random_state=RANDOM_SEED)
X_train_pc_drop, X_test_pc_drop, y_train_pc_drop, y_test_pc_drop = train_test_split(X_df_drop_reduced, y_drop_data, test_size=0.2, random_state=RANDOM_SEED)
X_train_no_stalk, X_test_no_stalk, y_train_no_stalk, y_test_no_stalk = train_test_split(X_scaled_no_stalk, y_no_stalk, test_size=0.2, random_state=RANDOM_SEED)

X_train_ohc_pc, X_test_ohc_pc, y_train_ohc_pc, y_test_ohc_pc = train_test_split(X_df_ohc_reduced, y_data, test_size=0.2, random_state=RANDOM_SEED)
X_train_ohc, X_test_ohc, y_train_ohc, y_test_ohc = train_test_split(pre_ohc_data, y_data, test_size=0.2, random_state=RANDOM_SEED)


 We will apply different classification models on our datasets without any tuning of the parameters. In this way we can
 see which datasets gives an overall efficient tradeoff between accuracy and time.
 In the next phase we will analyze one classifier at a time optimizing its parameters to obtain the best
 accuracy.

 Let's start defining the functions that we are going to use:

In [30]:

def print_gridcv_scores(grid_search, n=5):
    """
    Prints the best score achieved by a grid_search, alongside with its parametes

    :param (estimator) clf: Classifier object
    :param (int) n: Best n scores 
    """    
    
    t = PrettyTable()

    print("Best grid scores on validation set:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    
    t.field_names = ['Score'] + [f for f in params[0].keys()] 
    for mean, std, params in zip(means, stds, params):
        row=["%0.3f (+/-%0.03f)" % (mean, std * 2)] + [p for p in params.values()]
        t.add_row(row)
    print(t)
               

def param_tune_grid_cv(clf, params, X_train, y_train, cv, execution_time=False):
    """
    Function that performs a grid search over some parameters

    :param (estimator) clf: Classifier object
    :param (dictionary) params: parameters to be tested in grid search
    :param (array-like) X_train: List of data to be trained with
    :param (array-like) y_train: Target relative to X for classification or regression
    :param (cross-validation generator) cv: Determines the cross-validation splitting strategy
    """ 
    if execution_time:
      start = time.perf_counter()
    pipeline = Pipeline([('clf', clf)])
    grid_search = GridSearchCV(estimator=pipeline, 
                               param_grid=params, 
                               cv=cv, 
                               n_jobs=-1,       # Use all processors
                               scoring='f1',    # Use f1 metric for evaluation
                               return_train_score=True)
    grid_search.fit(X_train, y_train)
    if execution_time:
      end = time.perf_counter()
      return grid_search, "%.4f" % (end-start)
    return grid_search
   

def score(clfs, datasets):
    """
    Function that scores a classifier on some data
    
    :param (array of estimator) clf: Array of classifiers
    :param (dictionary) params: Dictionary of test data, passed like [(X_test, y_test)]

    """  
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))

    return scores


def hexToRGBA(hex, alpha):

    """
    Function that returns an rgba value from an hex and an opacity value
    
    :param (String) clf: Hex value 
    :param (float) params: Value between 0 and 1 indicating opacity

    """  

    r = int(hex[1:3], 16)
    g = int(hex[3:5], 16)
    b = int(hex[5:], 16)

    if alpha:
        return "rgba(" + str(r) + ", " + str(g) + ", " + str(b) + ", " + str(alpha) + ")"
    else:
        return "rgb(" + str(r) + ", " + str(g) + ", " + str(b) + ")"


def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.008, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, 
                                                            X, 
                                                            y, 
                                                            cv=cv, 
                                                            n_jobs=n_jobs, 
                                                            train_sizes=train_sizes, 
                                                            scoring="f1", 
                                                            random_state=RANDOM_SEED)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    # Prints lower bound (mean - std) of train 
    trace1 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean - train_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[0], 0.4),
        ),
    )
    # Prints upper bound (mean + std) of train
    trace2 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean + train_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[0], 0.4),
        ),
    )
    
    # Prints mean train score line
    trace3 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean, 
        showlegend=True,
        name="Train score",
        line = dict(
            color = PLOTLY_COLORS[0],
        ),
    )
    
    # Prints lower bound (mean - std) of test 
    trace4 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean - test_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[1], 0.4),
        ),
    )
        # Prints upper bound (mean + std) of test
    trace5 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean + test_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[1], 0.4),
        ),
    )

    # Prints mean test score line 
    trace6 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean, 
        showlegend=True,
        name="Test score",
        line = dict(
            color = PLOTLY_COLORS[1],
        ),
    )
    
    data = [trace1, trace2, trace3, trace4, trace5, trace6]
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)


def print_confusion_matrix(gs, X_test, y_test):

    """
    Function that prints confusion matrix for a classifier
    
    :param (estimator) clf: Classifier object
    :param (array-like) X_test: List of data to be tested with
    :param (array-like) y_test: List of labels for test 
    """  

    gs_score = gs.score(X_test, y_test)
    y_pred = gs.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    t = PrettyTable()
    t.add_row(["True Edible", cm[0][0], cm[0][1]])
    t.add_row(["True Poisonous", cm[1][0], cm[1][1]])
    t.field_names = [" ", "Predicted Edible", "Predicted Poisonous"]
    print(t)

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
    cm_df = pd.DataFrame(cm.round(3), index=["True edible", "True Poisonous"], columns=["Predicted edible", "Predicted poisonous"])
    cm_df


def print_raw_score(clf, X_test, y_test):
    """
    Function that scores a classifier on some data
    
    :param (array of estimator) clf: Array of classifiers
    :param (array-like) X_test: List of data to be tested with
    :param (array-like) y_test: List of labels for test 

    """  
    print("Score achieved by NB: %0.3f" % (score([clf], [(X_test, y_test)])[0]))


def plot_feature_importance(feature_importance, title):
    """
    Function that plots feature importance for a decision tree or a random forest classifier
    
    :param (dictionary) feature_importance: Dictionary of most important features sorted
    :param (str) title: Title of the plot

    """ 
    
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = PLOTLY_COLORS[0]),
        opacity=PLOTLY_OPACITY,
        name='Feature importance'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)


def print_performances(classifiers, classifier_names, auc_scores, X_test, y_test):
  
    """
    Function that scores a classifier on some data
    
    :param (array of estimator) clf: Array of classifiers
    :param (array-like) classifier_names: Title of the classifier
    :param (array-like) auc-score: Auc scores
    :param (array-like) X_test: List of data to be tested with
    :param (array-like) y_test: List of labels for test 

    """ 

    accs = []
    recalls = []
    precision = []
    results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
    for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
        y_pred = clf.predict(X_test)
        row = []
        row.append(accuracy_score(y_test, y_pred))
        row.append(precision_score(y_test, y_pred))
        row.append(recall_score(y_test, y_pred))
        row.append(f1_score(y_test, y_pred))
        row.append(auc)
        row = ["%.3f" % r for r in row]
        results_table.loc[name] = row
    return results_table



 Let's start defining the Stratified K-Folds cross-validator; it provides train/test indices to split data in train/test sets.

 This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of
 samples for each class. Stratification is generally a better scheme, both in terms of bias and variance, when compared to
 regular cross-validation.

 Then we define our classifiers, setting the `random_state` parameter to our usual seed value,
 in such a way that we can classify each time in the same way.

In [31]:

kf = StratifiedKFold(n_splits=5, random_state=RANDOM_SEED)
clf_nb = GaussianNB()
clf_knn = KNeighborsClassifier()
clf_rf = RandomForestClassifier(random_state=RANDOM_SEED)
clf_lr = LogisticRegression(random_state=RANDOM_SEED)
clf_svm = SVC(random_state=RANDOM_SEED)
clfs = [clf_nb, clf_knn, clf_rf, clf_lr, clf_svm]
TEST_PARAMS = {}


 We set `TEST_PARAMS` empty, because in this phase we are not interested in tuning parameters.

In [32]:
all_test_results = []
all_gss = []
times = np.zeros(7)
t = 0
      
for clf in clfs:

  gs_full, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
  times[0] = times[0] + float(t)
  
  gs_pc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pc, y_train_pc, kf, execution_time=True)
  times[1] = times[1] + float(t)
  
  gs_drop, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_drop, y_train_drop, kf, execution_time=True)
  times[2] = times[2] + float(t)
  
  gs_pc_drop, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pc_drop, y_train_pc_drop, kf, execution_time=True)
  times[3] = times[3] + float(t)
  
  gs_no_stalk, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_no_stalk, y_train_no_stalk, kf, execution_time=True)
  times[4] = times[4] + float(t)
  
  gs_ohc_pc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_ohc_pc, y_train_ohc_pc, kf, execution_time=True)
  times[5] = times[5] + float(t)
  
  gs_ohc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_ohc, y_train_ohc, kf, execution_time=True)
  times[6] = times[6] + float(t)
  
  
  gss = [gs_full, gs_pc, gs_drop, gs_pc_drop, gs_no_stalk, gs_ohc_pc, gs_ohc]
  all_gss.append(gss)
  test_results = score(gss, [(X_test, y_test), 
                             (X_test_pc, y_test_pc), 
                             (X_test_drop, y_test_drop), 
                             (X_test_pc_drop, y_test_pc_drop),
                             (X_test_no_stalk, y_test_no_stalk),
                             (X_test_ohc_pc, y_test_ohc_pc),
                             (X_test_ohc, y_test_ohc)])
  all_test_results.append(test_results)


 This is the score of the different classifiers on the different datasets.

In [33]:
dataset_strings = [" ", "full dataset", 
                   "dataset reduced on first 9 PC", 
                   "dataset with dropped missing values",
                   "dataset with dropped missing value reduced with first 9 PC",
                   "dataset with stalk-root field dropped",
                   "dataset ohc reduced on first 20 PC",
                   "dataset ohc"]

row_names = ["Naive Bayes", "KNN", "Random Forest", "Logistic Regression", "SVM"]

t = PrettyTable()
t.field_names = dataset_strings

all_rows = []
result_row = []

for name, results in zip(row_names, all_test_results):
  result_row.append(name)
  for r in results:
    result_row.append("%.3f" % r)
  all_rows.append(result_row)
  result_row = []
 
all_rows = sorted(all_rows, key=lambda kv: kv[1], reverse=True)

for k in all_rows:
    t.add_row(k)
    
print(t)

+---------------------+--------------+-------------------------------+-------------------------------------+------------------------------------------------------------+---------------------------------------+------------------------------------+-------------+
|                     | full dataset | dataset reduced on first 9 PC | dataset with dropped missing values | dataset with dropped missing value reduced with first 9 PC | dataset with stalk-root field dropped | dataset ohc reduced on first 20 PC | dataset ohc |
+---------------------+--------------+-------------------------------+-------------------------------------+------------------------------------------------------------+---------------------------------------+------------------------------------+-------------+
|         KNN         |    1.000     |             0.999             |                1.000                |                           0.999                            |                 1.000                 |        

 After have evaluated all these results, we are going to select the one that
 have the best tradeoff between time and score. Let's plot the mean score and the overall time taken to train all
 the different classifiers with respect to the same dataset.

In [34]:

means = np.mean(all_test_results, axis=0)
  
time_table = PrettyTable()
time_row = ["Total train time (s)"] + np.ndarray.tolist(times)
mean_row = ["Mean score for dataset"] + np.ndarray.tolist(means)
time_table.field_names = dataset_strings
time_table.add_row(time_row)
time_table.add_row(mean_row)
print(time_table)


+------------------------+--------------------+-------------------------------+-------------------------------------+------------------------------------------------------------+---------------------------------------+------------------------------------+-------------------+
|                        |    full dataset    | dataset reduced on first 9 PC | dataset with dropped missing values | dataset with dropped missing value reduced with first 9 PC | dataset with stalk-root field dropped | dataset ohc reduced on first 20 PC |    dataset ohc    |
+------------------------+--------------------+-------------------------------+-------------------------------------+------------------------------------------------------------+---------------------------------------+------------------------------------+-------------------+
|  Total train time (s)  |       5.8082       |             1.7562            |                2.1375               |                           1.0623                      

In [35]:
  
print("The dataset that gives the best overall performances is:")
print("\t- " + dataset_strings[means.argmax()] + ", with a score of " + str("%.3f" % means.max()))


The dataset that gives the best overall performances is:
	- dataset ohc reduced on first 20 PC, with a score of 0.988


 As we can see, the most accurate is the dataset encoded with `OneHotEncoder`; The training time though, is really high
 with respect to the other datasets. A good trade-off could be using the full dataset, because the accuracy is high and the
 training time is reasonable.

 But considering that the scores obtained are achieved without parameter tuning, I prefer to choose a "faster" dataset, even if the scores
 are not already optimal.
 From this point on, the classifiers will be tested on the full dataset with PCA. This because the cross-validation phase
 is almost four times faster, and the scores are similar.

 ### Logistic Regression
 The first classifier that we will analyze is the Logistic Regression classifier. It uses the
 sigmoid function to classify our samples:

 - $P(y=0 | X;\theta) = g(w^T X) = \frac{1}{1+e^{w^T X}}$
 - $P(y=1 | X;\theta) = 1 - g(w^T X) = \frac{e^{w^T X}}{1+e^{w^T X}}$


 This model, with respect to linear regression, can model better the zone close to
 0 and 1. To learn the weights, the $MLE$ is found and then the gradient descent algorithm
 is applied until the accuracy converges.

 They are:
 - `liblinear`. Solver, which is better for smaller datasets
 - `C`. Regularization strength, ranging from 0.01 to 100. A smaller value inidicates stronger regularization, like in svms.
 - `penalty`. "l1" and "l2" penalty for regularization, which are defined as:
   - l1, it penalizes every mistake at the same way
       - $S = \Sigma_{i=1}^{n}{|y_i - f(x_i)|}$

   - l2, it penalizes bigger values
       - $S = \Sigma_{i=1}^{n}{(y_i - f(x_i))^2}$

   - Where $y_i$ is the true label and $f(x_i)$ is the assigned label.

In [36]:

clf_lr = LogisticRegression(random_state=RANDOM_SEED)
gs_pc_lr = param_tune_grid_cv(clf_lr, LOGISTIC_REGRESSION_PARAMS, X_train_pc, y_train_pc, kf)
print_gridcv_scores(gs_pc_lr, n=5)


Best grid scores on validation set:
+------------------+--------+--------------+-------------+
|      Score       | clf__C | clf__penalty | clf__solver |
+------------------+--------+--------------+-------------+
| 0.879 (+/-0.009) |   1    |      l2      |  liblinear  |
| 0.879 (+/-0.009) |   1    |      l2      |    lbfgs    |
| 0.879 (+/-0.009) |   1    |      l2      |  newton-cg  |
| 0.879 (+/-0.009) |  100   |      l2      |    lbfgs    |
| 0.879 (+/-0.009) |  1000  |      l1      |  liblinear  |
+------------------+--------+--------------+-------------+


In [37]:
plot_learning_curve(gs_pc_lr.best_estimator_, "Learning curve of Logistic Regression", 
                    X_train_pc,
                    y_train_pc,
                    cv=5)


 From the graph we can see that the two lines approach themselves ans the test score
 tends to the train score. In this case, from the graph is evident that adding more
 samples is useless; we have more than enough, but the model is not complex enough to
 anchieve higher scores.

 From the learning curve we can see that, at the beginning

In [38]:
print_confusion_matrix(gs_pc_lr, X_test_pc, y_test_pc)


+----------------+------------------+---------------------+
|                | Predicted Edible | Predicted Poisonous |
+----------------+------------------+---------------------+
|  True Edible   |       779        |          80         |
| True Poisonous |       115        |         651         |
+----------------+------------------+---------------------+


 ### Support vector machine
 A Support Vector Machine (SVM) is a discriminative classifier formally defined by a
 separating hyperplane. In other words, given labeled training data (supervised learning),
 the algorithm outputs an optimal hyperplane which categorizes new examples. In two dimentional space
 this hyperplane is a line dividing a plane in two parts where in each class lay in either side.

 We perform the grid search over:
 - `linear`. This is the simplest SVM, finds the hyperplane which separates in the best way our samples.
 - `C`. The C parameter tells the SVM optimization how much you want to avoid misclassifying
 each training example. For large values of C, the optimization will choose a smaller-margin
 hyperplane if that hyperplane does a better job of getting all the training points
 classified correctly.
 - `rbf`. This parameter indicates that we are using a radial basis function kernel to perform the
 scalar product.
   - `gamma`. Defines how far the influence of a single training example reaches,
 with low values meaning ‘far’ and high values meaning ‘close’.

In [39]:
clf_svm = SVC(probability=True, random_state=RANDOM_SEED)
gs_pc_svm = param_tune_grid_cv(clf_svm, SVM_PARAMS, X_train_pc, y_train_pc, kf)
print_gridcv_scores(gs_pc_svm, n=5)


Best grid scores on validation set:
+------------------+--------+------------+-------------+
|      Score       | clf__C | clf__gamma | clf__kernel |
+------------------+--------+------------+-------------+
| 1.000 (+/-0.000) |   1    |     1      |     rbf     |
| 1.000 (+/-0.000) |  100   |     1      |     rbf     |
| 1.000 (+/-0.000) |  100   |    0.1     |     rbf     |
| 1.000 (+/-0.000) |   10   |     1      |     rbf     |
| 0.999 (+/-0.002) |   10   |    0.1     |     rbf     |
+------------------+--------+------------+-------------+


In [40]:
plot_learning_curve(gs_pc_svm.best_estimator_, "Learning curve of SVM", 
                    X_train_pc,
                    y_train_pc,
                    cv=5)


 We can see from the learning curve that we can achieve optimal performances
 before running out of training samples. After approximately 1300 samples analyzed,
 we already achieve a test score greater than 0.99 with very low standard deviation.

 This is the best score possibly achievable.
 We could expect it, but we will print anyway the confusion matrix:


In [41]:
print_confusion_matrix(gs_pc_svm, X_test_pc, y_test_pc)


+----------------+------------------+---------------------+
|                | Predicted Edible | Predicted Poisonous |
+----------------+------------------+---------------------+
|  True Edible   |       859        |          0          |
| True Poisonous |        0         |         766         |
+----------------+------------------+---------------------+


 ### Naive Bayes Classifier
 The naive bayes classifier is based on the Bayes theorem, which states that:

 - $P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

 Where:
 - $P(A)$, the prior, is the initial degree of belief in A.
 - $P(A|B)$, the posterior is the degree of belief having accounted for B.
 - $P(B|A$), is the likelyhood, the degree of belief in B, given that A is true.


 Using Bayes theorem, we can find the probability of A happening,
 given that B has occurred. Here, B is the evidence and A is the hypothesis. The assumption made
 here is that the predictors/features are independent. That is presence of one particular
 feature does not affect the other. Hence it is called naive.

In [42]:
clf_nb = GaussianNB()
gs_pc_nb = param_tune_grid_cv(clf_nb, TEST_PARAMS, X_train_pc, y_train_pc, kf)
print_gridcv_scores(gs_pc_nb, n=5)

print_confusion_matrix(gs_pc_nb, X_test_pc, y_test_pc)


Best grid scores on validation set:
+------------------+
|      Score       |
+------------------+
| 0.851 (+/-0.009) |
+------------------+
+----------------+------------------+---------------------+
|                | Predicted Edible | Predicted Poisonous |
+----------------+------------------+---------------------+
|  True Edible   |       777        |          82         |
| True Poisonous |       163        |         603         |
+----------------+------------------+---------------------+


In [43]:
plot_learning_curve(clf_nb, "Learning curve of GaussianNB", 
                    X_train, 
                    y_train, 
                    cv=5)


 To be a really simple classifier, it can achieve decent results. The independecy assumption
 does not work perfectly, because the different features are never completely independent from
 each other, but nevertheless it manages to achieve a good accuracy even if we used only
 the data projected on the principal components.

 With the naive bayes classifier, it is always
 better to use the original dataset, being really fast itself to train and to classify samples.
 With the ohc dataset, if we look in the upper table, we achieve a score around 0.945, which is
 a great score given our big assumption.

 ### Random Forest Classifier
 It is a tree-based method, derived from bagging decision trees, to which
 it is added another small trick which decorrelates the trees, in order to
 reduce further the variance.

 As in bagging, we build a number of decision trees on bootstrapped training
 samples. But when building these decision trees, each time a split in a tree
 is considered, a random selection of m predictors is chosen as split
 candidates from the full set of p predictors. The split is allowed to use only
 one of those m predictors, which typically are $m \approx \sqrt{p}$.

In [44]:

clf_pc_rf = RandomForestClassifier(random_state=RANDOM_SEED)
gs_pc_rf = param_tune_grid_cv(clf_pc_rf, RANDOM_FOREST_PARAMS, X_train_pc, y_train_pc, kf)
print_gridcv_scores(gs_pc_rf, n = 5)


Best grid scores on validation set:
+------------------+----------------+----------------+-------------------+-------------------+
|      Score       | clf__criterion | clf__max_depth | clf__max_features | clf__n_estimators |
+------------------+----------------+----------------+-------------------+-------------------+
| 0.996 (+/-0.002) |    entropy     |       50       |        log2       |        300        |
| 0.996 (+/-0.002) |    entropy     |      100       |        sqrt       |        300        |
| 0.996 (+/-0.002) |    entropy     |       75       |        log2       |        300        |
| 0.996 (+/-0.002) |    entropy     |      100       |        log2       |        300        |
| 0.996 (+/-0.002) |    entropy     |       75       |        sqrt       |        300        |
+------------------+----------------+----------------+-------------------+-------------------+


In [45]:
print_confusion_matrix(gs_pc_rf, X_test_pc, y_test_pc)


+----------------+------------------+---------------------+
|                | Predicted Edible | Predicted Poisonous |
+----------------+------------------+---------------------+
|  True Edible   |       858        |          1          |
| True Poisonous |        5         |         761         |
+----------------+------------------+---------------------+


In [46]:
plot_learning_curve(gs_pc_rf.best_estimator_, "Learning curve of Random Forest Classifier", 
                    X_train_pc,
                    y_train_pc,
                    cv=5)


 The training time is pretty high, but the accuracy as well. Looking at the learning curve, we can notice that
 the test error approaches the training error, but at a certain point starts to decrease. Analyzing over 1000 samples the
 accuracy does not improve any further. This may be due to some overfitting effect. A method to improve the accuracy could
 be using some additional regularization techniques, using for example `LightGMB` library to better fine-tune
 additional parameter.

 Now let's look deeper into the features of the Random Forest Classifier; let's see which of them weights more
 on the classification.

In [47]:
feature_importance = np.array(  sorted(zip(X_train_pc.columns, 
                                gs_pc_rf.best_estimator_.named_steps['clf'].feature_importances_),
                                key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance, "Feature importance in the random forest")


 Here we just see which are the components that weight more in the classification, but due to the
 fact that we reduced the dataset by means of PCA, we only see which component weighted more in
 the node splitting phase. Why the last components weight more than the first, the ones
 which contain most of the variability?

 The first principal component is a linear combination of all our features. The fact that it explains almost all the
 variability just means that most of the coefficients of the variables in the first principal component are significant.
 The classification trees we generate do binary splits on
 continuous variables that best separate the categories we want to classify. That is not exactly
 the same as finding orthogonal linear combinations of continuous variables that give the direction
 of greatest variance.

 If we would use the full dataset, we would see that the most important features are:

 1. `bruises`
 2. `gill-size`
 3. `gill-spacing`

 ### K-Nearest Neighbors Classifier
 K-NN is a type of instance-based learning, or lazy learning, where the function
 is only approximated locally and all computation is deferred until classification.
 The K-NN algorithm is among the simplest of all machine learning algorithms.

 The training phase of the algorithm consists only of storing the feature vectors
 and class labels of the training samples. In the classification phase, K is a user-defined constant,
 and an unlabeled vector (a query or test point) is classified by assigning the label which
 is most frequent among the k training samples nearest to that query point.

 The parameters for cross validation are:
 - `n_neighbors`. Number of closes samples to analyze
 - `weights`. Indicates the weight function to use in prediction.
   - `uniform`. All points in the neighborhood are weighted equally.
   - `distance`. Weight points by the inverse of their distance.
 In this case, closer neighbors of a query point will have a greater
 influence than neighbors which are further away.
 - `p`. Power parameter for the Minkowski metric (generalization of Euclidean distance)
   - p=1 uses `l1`
   - p=2 uses `l2`
   - p>2 minkowski_distance (l_p) is used.

In [48]:

clf_knn = KNeighborsClassifier()
gs_knn = param_tune_grid_cv(clf_knn, KNN_PARAMS, X_train_pc, y_train_pc, kf)
print_gridcv_scores(gs_knn, n=5)


Best grid scores on validation set:
+------------------+------------------+--------+--------------+
|      Score       | clf__n_neighbors | clf__p | clf__weights |
+------------------+------------------+--------+--------------+
| 1.000 (+/-0.000) |        2         |   2    |   distance   |
| 1.000 (+/-0.000) |        3         |   2    |   distance   |
| 1.000 (+/-0.001) |        3         |   2    |   uniform    |
| 1.000 (+/-0.001) |        2         |   10   |   distance   |
| 1.000 (+/-0.001) |        3         |   10   |   uniform    |
+------------------+------------------+--------+--------------+


In [49]:
print_confusion_matrix(gs_knn, X_train_pc, y_train_pc)


+----------------+------------------+---------------------+
|                | Predicted Edible | Predicted Poisonous |
+----------------+------------------+---------------------+
|  True Edible   |       3349       |          0          |
| True Poisonous |        0         |         3150        |
+----------------+------------------+---------------------+


In [50]:

plot_learning_curve(gs_knn.best_estimator_, "Learning curve of k-NN Classifier", 
                    X_train_pc,
                    y_train_pc,
                    cv=5)



 The accuracy achieved is really high.
 The K-NN classification with the whole dataset gives the same result but it takes more than
 7 times more time

 ### ROC curve
 At this point we need to evaluate the performances of the different classifiers and
 compare them.
 We are going to plot the ROC curve and the Area Under Curve for all our models.
 classifiers.
 The ROC curve is plotted with TPR against the FPR where TPR is on y-axis and FPR is on the x-axis.
 Specifically, these parameters are:

 - $TRP/Recall/Sensitivity = \frac{TP}{TP+FN}$
 - $FPR = \frac{FP}{TN+FP}$


 An excellent model has AUC near to the 1 which means it has good measure of
 separability. A poor model has AUC near to the 0 which means it has worst measure
 of separability.

In [51]:

def plot_roc_curve(classifiers, legend, title, X_test, y_test):
    t1 = go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        showlegend=False,
        mode="lines",
        name="",
        line = dict(
            color = COLOR_PALETTE[0],
        ),
    )
    
    data = [t1]
    aucs = []
    for clf, string, c in zip(classifiers, legend, COLOR_PALETTE[1:]):
        y_test_roc = np.array([([0, 1] if y else [1, 0]) for y in y_test])
        y_score = clf.predict_proba(X_test)
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(2):
            fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        aucs.append(roc_auc['micro'])

        trace = go.Scatter(
            x=fpr['micro'], 
            y=tpr['micro'], 
            showlegend=True,
            mode="lines",
            name=string + " (area = %0.2f)" % roc_auc['micro'],
            hoverlabel = dict(
                namelength=30
            ),
            line = dict(
                color = c,
            ),
        )
        data.append(trace)

    layout = go.Layout(
        title=title,
        autosize=False,
        width=550,
        height=550,
        yaxis=dict(
            title='True Positive Rate',
        ),
        xaxis=dict(
            title="False Positive Rate",
        ),
        legend=dict(
            x=0.4,
            y=0.06,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return aucs, iplot(fig, filename=title)


In [52]:

classifiers = [gs_pc_lr, gs_pc_svm, gs_pc_nb, gs_pc_rf, gs_knn]
classifier_names = ["Logistic Regression", "SVM", "GaussianNB", "Random Forest", "KNN"]
auc_scores, roc_plot = plot_roc_curve(classifiers, classifier_names, "ROC curve", X_test_pc, y_test_pc)
roc_plot


 Now let's look at some additional parameters that evaluate the goodness of our models:

 1. Accuracy
 2. Precision
       - $P = \frac{TP}{TP+FP}$
 3. Recall
       - $R = \frac{TP}{TP+FN}$
 4. F1 (weighted average of the precision and recall)
       - $F1= 2*\frac{P * R}{P + R}$
 5. Auc, area under the ROC curve

In [53]:
print_performances(classifiers, classifier_names, auc_scores, X_test_pc, y_test_pc)


,accuracy,precision,recall,f1,auc
Logistic Regression,0.880,0.891,0.850,0.870,0.911
SVM,1.000,1.000,1.000,1.000,1.000
GaussianNB,0.849,0.880,0.787,0.831,0.943
Random Forest,0.996,0.999,0.993,0.996,1.000
KNN,1.000,1.000,1.000,1.000,1.000


 The table shows that overall all classifiers performed well. The naive Bayes classifier
 is the one that performed worse, being the simplest one of them. Another reason is because
 we used the reduced dataset in classification; due to the fact that it is really fast,
 with the naive bayes is better to use the whole dataset. Infact
 the one OneHotEncoded anchieves a really high score, namely 0,945 (see score table in the choice
 of the dataset phase).
 The SVM and the K-NN are the classifiers that achieved the best score, hitting the 1 value on every parameter.
 The the Random Forest classifier achieved a similar score, even if slightly
 worse in accuracy and in recall.

 ## What NOT to do in the woods

 To have a little bit more fun we will try one more thing.
 If we are in a wood, how can we survive without the help of an SVM?
 Let's find out what are the peculiar traits of a poisonous mushroom

 Firstly, we create a KNN classifier and we iterate on all the columns,
 to see which of them gives a more accurate classification;
 In this way we can see which ones are the most important charateristics to
 classify a mushroom.

In [54]:

n_features = pre_ohc_data.shape[1]
clf = KNeighborsClassifier()
feature_score = []
t = PrettyTable()
t.field_names = ["Feature", "Score"]

for i in range(n_features):
    X_feature= np.reshape(pre_ohc_data.iloc[:,i:i+1],-1,1) # One column at a time
    scores = cross_val_score(clf, X_feature, y_data)
    feature_score.append(scores.mean())
    t.add_row([pre_ohc_data.columns[i], "{0:0.4f}".format(scores.mean())])

print(t)


+----------------------------+--------+
|          Feature           | Score  |
+----------------------------+--------+
|        cap-shape-c         | 0.5185 |
|        cap-shape-f         | 0.5289 |
|        cap-shape-k         | 0.5638 |
|        cap-shape-s         | 0.5180 |
|        cap-shape-x         | 0.4823 |
|       cap-surface-g        | 0.5180 |
|       cap-surface-s        | 0.5155 |
|       cap-surface-y        | 0.5059 |
|        cap-color-c         | 0.5180 |
|        cap-color-e         | 0.5028 |
|        cap-color-g         | 0.4947 |
|        cap-color-n         | 0.4927 |
|        cap-color-p         | 0.5142 |
|        cap-color-r         | 0.5180 |
|        cap-color-u         | 0.5180 |
|        cap-color-w         | 0.5180 |
|        cap-color-y         | 0.5515 |
|         bruises-t          | 0.5180 |
|           odor-c           | 0.5416 |
|           odor-f           | 0.7838 |
|           odor-l           | 0.5180 |
|           odor-m           | 0.5180 |


 Let's now select all the features that are more significant; we will
 pick the ones with a score greater than 0.7

In [55]:

f_importance = pd.Series(data = feature_score, index = pre_ohc_data.columns)
f_importance.sort_values(ascending=False, inplace=True)
f_importance[f_importance > 0.7]


odor-f                        0.783831
stalk-surface-above-ring-k    0.774481
stalk-surface-below-ring-k    0.766109
gill-size-n                   0.756319
spore-print-color-h           0.707000
dtype: float64

 Now we merge the unlabelled dataset with the labels, in such a way that
 we can group our samples using the class.

In [56]:

col_importance = f_importance[f_importance>0.7].index.values
pre_ohc_Xy = pd.concat([pre_ohc_data, pd.DataFrame(y_data, columns=['class'])], axis=1)
grouped = pre_ohc_Xy.groupby('class')


In [57]:
feat_edible = grouped.get_group(0)[col_importance].sum()
feat_edible

odor-f                          0.0
stalk-surface-above-ring-k    144.0
stalk-surface-below-ring-k    144.0
gill-size-n                   288.0
spore-print-color-h            48.0
dtype: float64

In [58]:
feat_poisonous = grouped.get_group(1)[col_importance].sum()
feat_poisonous


odor-f                        2160.0
stalk-surface-above-ring-k    2228.0
stalk-surface-below-ring-k    2160.0
gill-size-n                   2224.0
spore-print-color-h           1584.0
dtype: float64

 <a class="anchor-link" href="#conclusions">¶</a>
 ## Conclusions
 Our goal was to predict if a mushroom was poisonous or edible from its features.

 We understood that they are well separated and our classifiers can anchieve optimal performances.


 ![](https://infovisual.info/storage/app/media/01/img_en/024%20Mushroom.jpg)


 Looking at the KNN score using the different values for features, we understood that you should
 not eat a mushroom if it has:

 1. fishy odor
 2. stalk surface above ring silky
 3. stalk surface below ring silky
 4. gill size narrow
 5. spore print color chocolatey